In [3]:
import numpy as np
from transformers import AutoTokenizer
# from transformers import GPTNeoXForCausalLM
from gpt_neox_mlkv import GPTNeoXConfig, GPTNeoXForCausalLM
# from gpt_neox import GPTNeoXConfig, GPTNeoXForCausalLM
# check for cuda
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("using device:", device)
# load model
tokenizer = AutoTokenizer.from_pretrained('../pythia-160m-deduped_mlkv_6_6')
# tokenizer = AutoTokenizer.from_pretrained('../pythia-160m-deduped')
# config = GPTNeoXConfig.from_pretrained('../pythia-160m-deduped')
model = GPTNeoXForCausalLM.from_pretrained('../pythia-160m-deduped_mlkv_6_6')
# model = GPTNeoXForCausalLM.from_pretrained('../pythia-160m-deduped')
# tokenizer = AutoTokenizer.from_pretrained('EleutherAI/pythia-160m-deduped')
# model = GPTNeoXForCausalLM.from_pretrained('EleutherAI/pythia-160m-deduped')
# model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


using device: cuda


In [4]:
prompt = """The scaling of language models to tens even hundreds of billions of parameters made possible by the highly parallelizable transformer architecture has led to the rise of large language models (LLMs) as the pinnacle of natural language processing and language inference \parencite{pope2022efficiently}. Aside from achieving state-of-the-art in various benchmarks ranging from writing code to common sense and reasoning, LLMs have also been deployed massively at scale for applications such as OpenAI's ChatGPT or their API of various models \parencite{openai2023gpt4}. But serving billion-scale models to millions of customers becomes a major challenge, requiring massive compute to fulfil the needed latency constraints. Aside from a small amount reserved for computation, memory is excessively needed for two main things. Firstly the model weights, which can be large but stay fixed once deployed. The more concerning memory consumer is the key-value cache (KV cache), which contains the stored activations of attention modules at all previous time steps. This is the result of optimizing generative inference: tokens are generated one by one, auto-regressively. To avoid unnecessary re-computation of these activations, they are stored in memory to be used for generating future tokens. At small scales, KV cache can be unassuming, and memory is mostly needed for the weights. But taking OPT-175B as a large scale example, its 175 billion parameters require 325GB of memory. Yet when inferencing a sequence length of 2048 and computing a batch of 128 generations at once, the KV cache can take up to 950GB of memory \parencite{liu2023scissorhands}. In this case, the cache is three times larger than the model weights, yet even then, KV cache can still grow indefinitely in relation to the sequence length, batch size, and model dimension. TL;DR:"""
input_ids = tokenizer.encode(prompt, return_tensors='pt')
greedy_output = model.generate(input_ids, max_length=1000, temperature=0.5)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

C:\Users\Khalid Zuhri\AppData\Roaming\Python\Python311\site-packages\transformers\generation\configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
The scaling of language models to tens even hundreds of billions of parameters made possible by the highly parallelizable transformer architecture has led to the rise of large language models (LLMs) as the pinnacle of natural language processing and language inference \parencite{pope2022efficiently}. Aside from achieving state-of-the-art in various benchmarks ranging from writing code to common sense and reasoning, LLMs have also been deployed massively at scale for applications such as OpenAI's ChatGPT or their API of various models \parencite{openai2023gpt4}. But serving billion-scale models to millions of customers becomes a major challenge, requiring massive compute to fulfil the needed latency constraints. Aside from a small amount reserved for computation, memory is excessively needed for two main things. Firstly the model weights, which can be large but stay fixed once de

In [3]:
# inspect pytorch_model.bin and see all the keys
state_dict = torch.load('../pythia-160m-deduped/pytorch_model.bin', map_location='cpu')
for k in state_dict.keys():
    print(k)

gpt_neox.embed_in.weight
gpt_neox.layers.0.input_layernorm.weight
gpt_neox.layers.0.input_layernorm.bias
gpt_neox.layers.0.post_attention_layernorm.weight
gpt_neox.layers.0.post_attention_layernorm.bias
gpt_neox.layers.0.attention.bias
gpt_neox.layers.0.attention.masked_bias
gpt_neox.layers.0.attention.rotary_emb.inv_freq
gpt_neox.layers.0.attention.query_key_value.weight
gpt_neox.layers.0.attention.query_key_value.bias
gpt_neox.layers.0.attention.dense.weight
gpt_neox.layers.0.attention.dense.bias
gpt_neox.layers.0.mlp.dense_h_to_4h.weight
gpt_neox.layers.0.mlp.dense_h_to_4h.bias
gpt_neox.layers.0.mlp.dense_4h_to_h.weight
gpt_neox.layers.0.mlp.dense_4h_to_h.bias
gpt_neox.layers.1.input_layernorm.weight
gpt_neox.layers.1.input_layernorm.bias
gpt_neox.layers.1.post_attention_layernorm.weight
gpt_neox.layers.1.post_attention_layernorm.bias
gpt_neox.layers.1.attention.bias
gpt_neox.layers.1.attention.masked_bias
gpt_neox.layers.1.attention.rotary_emb.inv_freq
gpt_neox.layers.1.attention.q

In [4]:
state_dict['gpt_neox.layers.11.attention.query_key_value.weight'].shape

torch.Size([2304, 768])